In [1]:
!pip install tensorflow tensorflow-datasets

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

In [3]:
(ds_train, ds_test), ds_info = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:]'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/incomplete.BFMRGD_4.0.1/cats_vs_dogs-train.tfrecord*...:   0%…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.1. Subsequent calls will reuse this data.


In [4]:
IMG_SIZE = 128

def format_image(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = image / 255.0
    return image, label

BATCH_SIZE = 32

ds_train = ds_train.map(format_image).cache().shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.map(format_image).cache().batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [5]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(ds_train, validation_data=ds_test, epochs=5)

Epoch 1/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 549s 939ms/step - accuracy: 0.8718 - loss: 0.3019 - val_accuracy: 0.8233 - val_loss: 0.3998
Epoch 2/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 541s 930ms/step - accuracy: 0.9032 - loss: 0.2311 - val_accuracy: 0.8291 - val_loss: 0.4326
Epoch 3/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 561s 929ms/step - accuracy: 0.9377 - loss: 0.1575 - val_accuracy: 0.8403 - val_loss: 0.4853
Epoch 4/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 545s 936ms/step - accuracy: 0.9549 - loss: 0.1177 - val_accuracy: 0.8334 - val_loss: 0.6168
Epoch 5/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 558s 928ms/step - accuracy: 0.9738 - loss: 0.0697 - val_accuracy: 0.8332 - val_loss: 0.6611


In [8]:
loss, acc = model.evaluate(ds_test)
print(f"Precisión de las prueba: {acc:.2f}")

146/146 ━━━━━━━━━━━━━━━━━━━━ 36s 247ms/step - accuracy: 0.8307 - loss: 0.6641
Precisión de las prueba: 0.83
